# Projet Deep Reinforcement Learning - M2 IABD

Avant toute chose, vérifiez que vous avez bien un GPU (sous Colab : Exécution > Modifier le type d'exécution > Accélérateur matériel)

Puis lancez la cellule suivante qui va connecter votre google drive sous Colab et définir le dossier de travail.

In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/gdrive')
    import os
    os.makedirs('/gdrive/My Drive/IABD-DeepRL', exist_ok=True)
    data_dir = '/gdrive/My Drive/IABD-DeepRL'
else:
    data_dir = '.'
%cd $data_dir


Lancez ensuite la cellule suivante qui met à jour le dépôt et installe les dépendances. Cette cellule peut prendre un certain temps à s'exécuter (surtout la 1ère fois) :

In [ ]:
!git clone https://github.com/pcouy/ESGI-M2-IABD
!cd ESGI-M2-IABD && git pull
!pip install -U pip
!pip install ESGI-M2-IABD/code/

Le fait de choisir un dossier de travail dans votre Drive vous permet de rendre persistantes les modifications que vous apporterez au code fourni.

La cellule suivante fournit quelques fonctions utilitaires qui sont utilisées pour afficher les vidéos des épisodes de test dans le *notebook* :

In [ ]:
# Fonctions utilitaires pour afficher interactivement les vidéos des épisodes de test
# ATTENTION : Le slider de view_videos ne fonctionne que s'il n'y a pas d'exécution déjà en cours !
import os, io, glob, base64, threading
from ipywidgets import interact, widgets
from IPython.display import HTML, Image, display

def load_and_display(path):
    """Show a video at `path` within IPython Notebook."""
    if not os.path.isfile(path):
        raise NameError("Cannot access: {}".format(path))

    video = io.open(path, "r+b").read()
    encoded = base64.b64encode(video)

    display(HTML(
        data="""
        <video alt="test" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
        </video>
        """.format(encoded.decode("ascii"))
    ))

def show_last_video(agent_save_dir):
    dirname = os.path.join(agent_save_dir, "videos")
    files = sorted([f for f in os.listdir(dirname)])
    file = files[-1]
    path = os.path.join(dirname, file, "rl-video-episode-0.mp4")
    load_and_display(path)

def view_videos(agent):
    dirname = os.path.join(agent.save_dir, "videos")
    files = sorted([f for f in os.listdir(dirname)])
    print(files)
    N = len(files)
    def d(file: str) -> None:
        path = os.path.join(dirname, file, "rl-video-episode-0.mp4")
        load_and_display(path)
    interact(d, file=widgets.SelectionSlider(options=files, value=files[-1]))

## Objectif du projet

L'objectif de ce projet est la mise en oeuvre des modèles de *Deep Reinforcement Learning* vus en cours.

Vous utiliserez dans un premier temps le code fourni pour créer plusieurs agents et comparer leurs performances.

La seconde partie de ce travail consitera à reproduire vous-même les résultats d'un ou plusieurs articles scientifiques de votre choix en vous appuyant sur le code fourni et les notions vues en cours.

Pour ces deux parties, vous devrez produire un compte rendu présentant les résultats de vos expériences, accompagné du code permettant de reproduire vos résultats.

Pour l'ensemble du projet, vous évaluerez vos agent sur l'environnement *Snake* 8x8. Bien qu'il soit possible d'appliquer les algorithmes vus ici à d'autres environnements plus complexes, il est recommandé de s'en tenir au jeu *Snake* afin d'obtenir des résultats en quelques heures (les environnements *Atari* nécessitant par exemple plusieurs jours d'entrainement).

### Conseils

* Testez vos implémentations sur des problèmes plus simples (le *Snake* 4x4 par exemple) pour gagner du temps sur le débuggage
* Testez les valeurs des hyper-paramètres en partant des valeurs données dans les articles scientifiques de référence, et ajustez les progressivement. Le code fourni enregistre les résultats de chaque expérience, accompagné des hyper-paramètres utilisés. Utilisez cet historique à votre avantage pour accéler votre recherche.
* Répartissez bien les tâches entre les membres de votre groupe. Chaque expérience coûte un certain temps, ne le gaspillez pas en re-faisant des expériences déjà tentées par vos collègues.
* Google limite malheureusement le temps de GPU par compte sur Colab. Vous pouvez contourner cette limite si vos disposez de plusieurs comptes Google
* Il peut parfois être utile de surveiller l'évolution d'une expérience pour pouvoir l'arrêter prématurément lorsque celle ci est manifestement un échec. Attention toutefois à ne pas perdre des heures à surveiller l'apprentissage de votre agent.
* Vous n'aurez pas le temps de trouver vous-même de bonnes valeurs pour tous les hyper-paramètres, c'est un processus très lent. N'hésitez pas à utiliser toutes les ressources que vous pourrez trouver en ligne pour vous y aider.

## Introduction - DQN Simple

La cellule ci-dessous créé une classe héritant de 3 autres, qui ajoute à la fois le *replay* d'expérience et la fonction de valeur cible au `QLearningAgent`.

Ce sont ces modifications qui permettent d'utiliser un réseau de neurone pour approximer la fonction de valeur.

In [ ]:
import code_tp as TP
from code_tp import agents, value_functions, policies
from code_tp import wrappers
import gym

# La déclaration de classe ci-dessous permet de combiner dans
# la classe `DQNAgent` les amélirations apportées par les 3
# classes dont elle hérite. Notez particulièrement la section
# `Method resolution order` dans la sortie de cette cellule.
class DQNAgent(agents.target_value.TargetValueAgent, agents.replay_buffer.ReplayBufferAgent, agents.base.QLearningAgent):
    pass
help(DQNAgent)

Nous pouvons mantenant créer l'environnement *Snake* et instancier l'agent. Les valeurs des paramètres donnés ci-dessous ne sont *a priori* pas optimaux mais fonctionnent bien pour le *Snake* en pratique.

In [ ]:
from torch import nn

env = gym.make("Snake-8x8-v0")
env = gym.wrappers.FrameStack(env,2)
env = wrappers.utils.TimeToChannels(env)
env = wrappers.utils.BoredomWrapper(env, -0.0005)

a=TP.create_agent_from_env(env,
agent_class=DQNAgent,
value_class=value_functions.neural.ConvolutionalQFunction,
policy_class=policies.greedy.EGreedyPolicy,
agent_args={
    'gamma':0.99, 'replay_buffer_class':agents.replay_buffer.ReplayBuffer,
    'replay_buffer_args': {
        'batch_size':32,
        'max_size':150000
    },
    'target_update': 1000,
    'update_interval': 1,
    'save_dir': os.path.join(data_dir, "results")+"/"
},
value_args={
    'lr':1e-3,
    'nn_args': {
        'kernel_size':[1,3,3,3],
        'stride':1,
        'n_filters':[4,16,32,64],
        'padding':1,
        'pooling':[None,None,None,None],
        'activation': nn.ReLU,
        'output_stack_class': value_functions.neural_nets.linear.LinearNeuralStack,
        'output_stack_args': {
            'layers': [256]
        }
    }
},
policy_args={
    'greedy_policy_class': policies.greedy.GreedyQPolicy,
    'epsilon': 0.9, 'epsilon_decay': 2e-5, 'epsilon_min': 0.02,
    'epsilon_test':0
})


On charge à présent [*Tensorboard*](https://www.tensorflow.org/tensorboard) qui est un outil permettant de visualiser les courbes d'entrainement en direct et de comparer les résultats de plusieurs expériences :

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results --samples_per_plugin scalars=50000

Les courbes de vos entrainements passés et en cours s'afficheront dans Tensorboard. Vous pouvez sélectionner les expériences que vous souhaitez afficher. L'onglet "Time Series" a l'avantage de mettre à jour automatiquement l'échelle des courbes pour afficher les nouvelles données.

Il est recommandé d'utiliser une valeur de 0.99 pour le paramètre "smoothing" dans Tensorboard.

Il reste ensuite à entrainer l'agent.

In [ ]:
a.train(200000, 200, test_callbacks=[show_last_video])

## À vous de jouer !

Comparez les résultats obtenus par le DQN avec les résultats obtenus en intégrant diverses avancées en matière de *Deep Reinforcement Learning*. Inspirez vous des méthodes d'évaluation utilisées dans la littérature scientifique pour évaluer les résultats de vos propres expériences.

Le code fourni propose déjà des implémentations de plusieurs améliorations proposées dans les articles scientifiques suivants :

* [Hasselt, Hado van, Arthur Guez, et David Silver. « Deep Reinforcement Learning with Double Q-learning », 8 décembre 2015](http://arxiv.org/abs/1509.06461)
* [Schaul, Tom, John Quan, Ioannis Antonoglou, et David Silver. « Prioritized Experience Replay », 25 février 2016](http://arxiv.org/abs/1511.05952)
* [Wang, Ziyu, Tom Schaul, Matteo Hessel, Hado van Hasselt, Marc Lanctot, et Nando de Freitas. « Dueling Network Architectures for Deep Reinforcement Learning », 5 avril 2016](http://arxiv.org/abs/1511.06581)

Vous devez au minimum :

* Réaliser des expériences mettant en avant l'impact individuel de chacune de ces innovations par rapport au DQN fourni ci-dessus. Vous devrez trouver vous-même les classes implémentant ces techniques dans le code fourni (en utilisant par exemple la documentation qui l'accompagne)
* Implémenter une innovation présentée dans un article scientifique de votre choix dans une nouvelle classe compatible avec le code fourni. Vous réaliserez une expérience mettant en avant l'impact de cette amélioration par rapport au DQN.

Vous pourrez ensuite combiner plusieurs de ces innovations et constater leur impact sur les politiqes apprises par vos agents.

### Lectures utiles

#### Divers

>Sutton, Richard S., et Andrew G. Barto. Reinforcement Learning: An Introduction. Second edition. Adaptive Computation and Machine Learning Series. Cambridge, Massachusetts: The MIT Press, 2018. http://incompleteideas.net/book/RLbook2018.pdf
>
> >Ce livre, bien que ne traitant pas spécifiquement de **Deep** Reinforcement Learning, est un référénce sur le sujet de l'apprentissage par renforcement et présente un grand nombre de techniques.

> Deepmind. « Deep Reinforcement Learning ». Consulté le 22 septembre 2021. https://deepmind.com/blog/article/deep-reinforcement-learning.
>
> > Article de blog de DeepMind datant de 2016 dressant un état de l'art du *Deep Reinforcement Learning* (qui était encore un domaine très jeune) à l'époque. Ce blog est par ailleurs un bon moyen de se tenir informé des derniers travaux d'un des laboratoires les plus prolifiques sur le sujet.




#### Articles scientifiques

Les articles suivants proposent pour la plupart au moins une inovation en terme de Deep Reinforcement Leaning appliquable aux techniques vues en cours. Certains d'entre eux se consacrent à dresser un état de l'art ou à réaliser des études comparatives entre plusieurs techniques.

* Andrychowicz, Marcin, Filip Wolski, Alex Ray, Jonas Schneider, Rachel Fong, Peter Welinder, Bob McGrew, Josh Tobin, Pieter Abbeel, et Wojciech Zaremba. « Hindsight Experience Replay ». arXiv:1707.01495 [cs], 23 février 2018. http://arxiv.org/abs/1707.01495.
 Burda, Yuri, Harri Edwards, Deepak Pathak, Amos Storkey, Trevor Darrell, et Alexei A. Efros. « Large-Scale Study of Curiosity-Driven Learning ». arXiv:1808.04355 [cs, stat], 13 août 2018. http://arxiv.org/abs/1808.04355.
* Christiano, Paul, Jan Leike, Tom B. Brown, Miljan Martic, Shane Legg, et Dario Amodei. « Deep reinforcement learning from human preferences ». arXiv:1706.03741 [cs, stat], 13 juillet 2017. http://arxiv.org/abs/1706.03741.
* Fortunato, Meire, Mohammad Gheshlaghi Azar, Bilal Piot, Jacob Menick, Ian Osband, Alex Graves, Vlad Mnih, et al. « Noisy Networks for Exploration ». arXiv:1706.10295 [cs, stat], 9 juillet 2019. http://arxiv.org/abs/1706.10295.
* Hasselt, Hado van, Arthur Guez, et David Silver. « Deep Reinforcement Learning with Double Q-learning ». arXiv:1509.06461 [cs], 8 décembre 2015. http://arxiv.org/abs/1509.06461.
* Hessel, Matteo, Joseph Modayil, Hado van Hasselt, Tom Schaul, Georg Ostrovski, Will Dabney, Dan Horgan, Bilal Piot, Mohammad Azar, et David Silver. « Rainbow: Combining Improvements in Deep Reinforcement Learning ». arXiv:1710.02298 [cs], 6 octobre 2017. http://arxiv.org/abs/1710.02298.
* Hessel, Matteo, Joseph Modayil, Hado van Hasselt, Tom Schaul, Georg Ostrovski, Will Dabney, Dan Horgan, Bilal Piot, Mohammad Azar, et David Silver. « Rainbow: Combining Improvements in Deep Reinforcement Learning ». In Thirty-Second AAAI Conference on Artificial Intelligence, 2018. https://www.aaai.org/ocs/index.php/AAAI/AAAI18/paper/view/17204.
* Li, Yuxi. « Deep Reinforcement Learning ». arXiv:1810.06339 [cs, stat], 15 octobre 2018. http://arxiv.org/abs/1810.06339.
* Machado, Marlos, Marc Bellemare, Erik Talvitie, Joel Veness, Matthew Hausknecht, et Michael Bowling. « Revisiting the Arcade Learning Environment: Evaluation Protocols and Open Problems for General Agents (Extended Abstract) », 5573‑77, 2018. https://doi.org/10.24963/ijcai.2018/787.
* Mnih, Volodymyr, Koray Kavukcuoglu, David Silver, Alex Graves, Ioannis Antonoglou, Daan Wierstra, et Martin Riedmiller. « Playing Atari with Deep Reinforcement Learning ». arXiv:1312.5602 [cs], 19 décembre 2013. http://arxiv.org/abs/1312.5602.
* Mnih, Volodymyr, Koray Kavukcuoglu, David Silver, Andrei A. Rusu, Joel Veness, Marc G. Bellemare, Alex Graves, et al. « Human-Level Control through Deep Reinforcement Learning ». Nature 518, nᵒ 7540 (26 février 2015): 529‑33. https://doi.org/10.1038/nature14236.
* Osband, Ian, Charles Blundell, Alexander Pritzel, et Benjamin Van Roy. « Deep Exploration via Bootstrapped DQN ». arXiv:1602.04621 [cs, stat], 4 juillet 2016. http://arxiv.org/abs/1602.04621.
*  Pathak, Deepak, Pulkit Agrawal, Alexei A. Efros, et Trevor Darrell. « Curiosity-Driven Exploration by Self-Supervised Prediction ». In 2017 IEEE Conference on Computer Vision and Pattern Recognition Workshops (CVPRW), 488‑89. Honolulu, HI, USA: IEEE, 2017. https://doi.org/10.1109/CVPRW.2017.70.
* Plappert, Matthias, Rein Houthooft, Prafulla Dhariwal, Szymon Sidor, Richard Y. Chen, Xi Chen, Tamim Asfour, Pieter Abbeel, et Marcin Andrychowicz. « Parameter Space Noise for Exploration ». arXiv:1706.01905 [cs, stat], 31 janvier 2018. http://arxiv.org/abs/1706.01905.
* Schaul, Tom, John Quan, Ioannis Antonoglou, et David Silver. « Prioritized Experience Replay ». arXiv:1511.05952 [cs], 25 février 2016. http://arxiv.org/abs/1511.05952.
* Schmid, Martin, Matej Moravcik, Neil Burch, Rudolf Kadlec, Josh Davidson, Kevin Waugh, Nolan Bard, et al. « Player of Games ». arXiv:2112.03178 [cs], 6 décembre 2021. http://arxiv.org/abs/2112.03178.
Sutton, Richard S. « Learning to Predict by the Methods of Temporal Differences ». Machine Learning 3, nᵒ 1 (août 1988): 9‑44. https://doi.org/10.1007/BF00115009.
* Vecerik, Mel, Todd Hester, Jonathan Scholz, Fumin Wang, Olivier Pietquin, Bilal Piot, Nicolas Heess, Thomas Rothörl, Thomas Lampe, et Martin Riedmiller. « Leveraging Demonstrations for Deep Reinforcement Learning on Robotics Problems with Sparse Rewards ». arXiv:1707.08817 [cs], 8 octobre 2018. http://arxiv.org/abs/1707.08817.
* Wang, Ziyu, Tom Schaul, Matteo Hessel, Hado van Hasselt, Marc Lanctot, et Nando de Freitas. « Dueling Network Architectures for Deep Reinforcement Learning ». arXiv:1511.06581 [cs], 5 avril 2016. http://arxiv.org/abs/1511.06581.
